# Quantified Degree of Belief, Posterior Distribution

Once the client has checked that the model assumptions are met using **Posterior Predictive Checks**, they can interpret the results of estimation and forecasting. 

Once they have settled on their final model (!! after **Model Selection**, which we will discuss in a later notebook), we want them to correctly assess it.

The client will have access to point estimates for model parameters and forecasts.

These are based on the mode of the parameters' posterior distribution.  

The client will have access to measures of variability for the parameter estimates and forecast.  Finally, we will provide the client with tools to assess the results with statistical confidence, a quantified degree of belief.  These can include statistical tests and confidence intervals for the results. 

In this notebook we will discuss what we will make available and justify why.

We will focus on model parameter estimation.  What we do with forecasting can be informed by what we discuss here, but those details need to be investigated.

For the moment, we are not using MCMC directly to estimate model parameters.  MCMC is being using for posterior predictive checks and statistical certification.

For model estimation, MCMC can be a more computationally intensive alternative to what we discuss here.  In small sample size situations, MCMC can be much more accurate as well.

## Normal Approximation to the Posterior

We will use numerical optimization to obtain the posterior mode $\widehat{\boldsymbol \theta}$, maximizing the posterior

$\displaystyle \pi(\boldsymbol{\theta}\vert {\bf x})$

The posterior is proportional (where the scaling does not depend on $\boldsymbol{\theta}$) to the prior and likelihood (or density of the data).


$\displaystyle \pi(\boldsymbol{\theta}\vert {\bf x}) \propto L(\boldsymbol{\theta}\vert {\bf x}) \pi(\boldsymbol{\theta})$

As in maximum likelihood, we directly maximize the log-posterior, $\log \pi(\boldsymbol{\theta}\vert {\bf x})$ because it is more stable.

Now, as described in section 4.1 of BDA 3 (Bayesian Data Analysis 3rd edn A. Gelman, J. B. Carlin, H. S. Stern, D. B. Dunson, A. Vehtari and D. B. Rubin, 2013 Boca Raton, Chapman and Hall–CRC), we can approximate $\ln \pi(\boldsymbol{\theta}\vert {\bf x})$ using a 2nd order Taylor Expansion around $\widehat{\boldsymbol \theta}$. 

$\displaystyle \log \pi(\boldsymbol{\theta}\vert {\bf x}) \approx \log \pi(\widehat{\boldsymbol \theta}\vert{\bf x}) + (\boldsymbol{\theta} - \widehat{\boldsymbol \theta} )^TS({\boldsymbol \theta})\vert_{{\boldsymbol \theta}={\widehat{\boldsymbol \theta}}} + \frac{1}{2}(\boldsymbol{\theta} - \widehat{\boldsymbol \theta} )^T H(\widehat{\boldsymbol \theta}) (\boldsymbol{\theta} - \widehat{\boldsymbol \theta} )$  

Where $S(\boldsymbol{\theta})$ is the score function

$\displaystyle S(\boldsymbol{\theta}) = \frac{\delta}{\delta \boldsymbol{\theta}} \log \pi(\boldsymbol{\theta}\vert {\bf x})$

and $H(\boldsymbol{\theta})$ is the Hessian function.

$\displaystyle H(\boldsymbol{\theta}) =  \frac{\delta}{\delta \boldsymbol{\theta}^T} S(\boldsymbol{\theta}) $

We assume that $\widehat{\boldsymbol \theta}$ is in the interior of the parameter space (or support) of $\boldsymbol{\theta}$.  

Also, $\pi(\boldsymbol{\theta}\vert {\bf x})$ is a continuous function of $\boldsymbol{\theta}$.

Finally the Hessian matrix, $H(\boldsymbol{\theta})$ is negative definite, so $-H(\boldsymbol{\theta})$ is positive definite.  This means that we can invert $-H(\boldsymbol{\theta})$ and get a matrix that is a valid covariance.

With these assumptions, as the sample size $n\to\infty$ the quadratic approximation for $\log \pi(\boldsymbol{\theta}\vert {\bf x})$ becomes more accurate.  At the posterior mode ${\boldsymbol \theta}={\widehat{\boldsymbol \theta}}$, 
$\log \pi(\boldsymbol{\theta}\vert {\bf x})$ is maximized and $0=S({\boldsymbol \theta})\vert_{{\boldsymbol \theta}={\widehat{\boldsymbol \theta}}}$.

Given this, we can exponentiate the approximation to get


$\displaystyle \pi(\boldsymbol{\theta}\vert {\bf x}) \approx \pi(\widehat{\boldsymbol \theta}\vert{\bf x}) \exp(\frac{1}{2} (\boldsymbol{\theta} - \widehat{\boldsymbol \theta} )^T H(\widehat{\boldsymbol \theta}) (\boldsymbol{\theta} - \widehat{\boldsymbol \theta} ))$

So for large $n$, the posterior distribution of ${\boldsymbol \theta}$ is approximately proportional to a multivariate normal density with mean $\widehat{\boldsymbol{\theta}}$ and covariance $-H(\widehat{\boldsymbol{\theta}})^{-1}$.  

$\displaystyle {\boldsymbol \theta} \vert x \approx_D N(\widehat{\boldsymbol{\theta}}, -H(\widehat{\boldsymbol{\theta}})^{-1})$ 

Another caveat for this result is that the prior should be proper, or at least lead to a proper posterior.  
Our asymptotic results are depending on probabilities integrating to 1.


## Parameter Constraints and Transformations

Optimization can be easier if the parameters are defined over the entire real line.  

Parameters that do not follow this rule are plentiful.  Variances are only positive.  Probabilities are in [0,1].

Stan provides the ease of optimizing parameters over the entire real line by creating unconstrained parameters.
These are continuous functions of the constrained parameters, which may be defined on intervals of the real line.
 

For example, the unconstrained version of a standard deviation parameter $\sigma$ is $\psi= \log \sigma$. $\psi$ is defined over the entire real line.      


It will be useful for us to consider the constrained parameters as being functions of the unconstrained parameters.  

So $\sigma=exp(\psi)$ is our constrained parameter of $\psi$.

 So the posterior mode of the constrained parameters  ${\boldsymbol{\theta_c}}$  is $\widehat{\boldsymbol \theta}_{\boldsymbol c} = g(\widehat{\boldsymbol \theta})$.  We will call $g$ the **constraint** function

Then we can use the delta method on $g$.  

A first-order taylor approximation of $g({\boldsymbol \theta})$ at $\widehat{\boldsymbol \theta}$ yields 

$\displaystyle g({\boldsymbol \theta}) \approx g(\widehat{\boldsymbol \theta}) + \left\{\frac{\delta}{\delta \widehat{\boldsymbol{\theta}}} g(\widehat{\boldsymbol{\theta}})\right\} ({\boldsymbol \theta} - \widehat{\boldsymbol{\theta}})$

Remembering that $\boldsymbol \theta$ is approximately normal, the rules about linear transformations for multivariate normal random vectors tell us that

$\displaystyle {\boldsymbol{\theta_c}}\vert x = g({\boldsymbol \theta}) \vert x \approx_D N \left\lbrack g(\widehat{\boldsymbol{\theta}}), \left\{\frac{\delta}{\delta \widehat{\boldsymbol{\theta}}} g(\widehat{\boldsymbol{\theta}})\right\}^T \left\{-H(\widehat{\boldsymbol{\theta}})^{-1}\right\} \left\{\frac{\delta}{\delta \widehat{\boldsymbol{\theta}}} g(\widehat{\boldsymbol{\theta}})\right\}\right\rbrack$ 

This involved a first-order approximation of $g$.  

Earlier we used a second order approximation for taking the numeric derivative.  Why would we just do a first-order here?    

Traditionally the delta-method is taught and used as only a first-order method.  

Usually the functions used in the delta method are not incredibly complex.  It is *good enough* to to use the first-order approximation.  

## Hessian approximation

To be able to use the normal approximation, we need $\widehat{\boldsymbol{\theta}}$, $H(\widehat{\boldsymbol{\theta}})^{-1}$, and $\frac{\delta}{\delta \widehat{\boldsymbol{\theta}}} g(\widehat{\boldsymbol{\theta}})$.

As mentioned before, we use numerical optimization to get $\widehat{\boldsymbol{\theta}}$.  Ideally, we would have analytic expressions for $H$ and the derivatives of $g$.

In PyStan, no estimate of the Hessian is exposed.  

RStan estimates a Hessian using numerical differentiation on the analytic gradient.  See:

https://stackoverflow.com/questions/27202395/how-do-i-get-standard-errors-of-maximum-likelihood-estimates-in-stan

https://cran.r-project.org/web/packages/rstan/rstan.pdf

We can also perform numerical differentiation in Python to get the Hessian and the gradient of the constraint function $g$.  This will be less accurate than an analytic expression, 
and will also normally be more computationaly intensive.

But "*Once you learn how to take one numeric derivative, you can take the numeric derivative of anything*".  So using numerical differentiation is a very flexible technique that we can easily apply to all the models that we would use in Stan.

## Numerical Differentiation

So numeric derivatives can be very pragmatic, and flexible.

How do you compute them?  Are they accurate?  We use the following reference as a guide.

William H. Press, Saul A. Teukolsky, William T. Vetterling, and Brian P. Flannery. 2007. Numerical Recipes 3rd Edition: The Art of Scientific Computing (3rd. ed.). Cambridge University Press, USA.

Let's look at the definition of a derivative.

$f'(x) = \displaystyle\lim_{h\to0}\frac{f(x+h)-f(x)}{h}$

To approximate $f'(x)$ numerically, couldn't we just plugin a small value for $h$ and compute the scaled difference? 

Yes.  And that is basically what happens.  

We do do a little more work to choose $h$ and use a second-order approximation instead of a first-order.  

We can see that the scaled difference is a first-order approximation by looking at the Taylor series expansion around $x$.

Taylor's theorem with remainder gives 

$f(x+h)$

 
$ = f(x) + ((x+h)-x)f'(x) + .5((x+h)-x)^2 f''(\epsilon)$

$ = f(x) + -h f'(x) + .5 h f''(\epsilon)$

where $\epsilon$ is between $x$ and $x+h$.

We can rearrange to get 


$\displaystyle\frac{f(x+h)-f(x)}{h} - f'(x) = .5 h f''(\epsilon)$

This is linear in h.

We can do second-order approximations for $f(x+h)$ and $f(x-h)$

$\displaystyle f(x+h) = f(x) + ((x+h)-x)f'(x) + \frac{((x+h)-x)^2 f''(x)}{2!} + \frac{((x+h)-x)^3f'''(\epsilon_1)}{3!}$

$\displaystyle f(x-h) = f(x) + ((x-h)-x)f'(x) + \frac{((x-h)-x)^2 f''(x)}{2!} + \frac{((x-h)-x)^3f'''(\epsilon_2)}{3!}$

where $\epsilon_1$ is between $x$ and $x+h$ and $\epsilon_2$ is between $x-h$ and $x$.

Then we have

$\displaystyle \frac{f(x+h) - f(x-h)}{2h} - f'(x) = h^2 \frac{f'''(\epsilon_1)+ f'''(\epsilon_2)}{12}$

This is quadratic in h.

The first term takes equal input from both sides of $x$, so we call it a centered derivative.

So we choose a small value of $h$ and plug it into 

$\displaystyle \frac{f(x+h) - f(x-h)}{2h}$

to approximate $f'(x)$.

Our derivation used a single input function $f$.  The idea applies to partial derivatives of multi-input functions as well.  The inputs that you aren't taking the derivative with respect to are treated as fixed parts of the function.   

### Choosing a Bandwidth, *h*

In practice, second order approximation actually involves two sources of error.

**Roundoff error**, $\epsilon_r$ arises from being unable to represent $x$ and $h$ or functions of them with exact binary represetation.

$\displaystyle \epsilon_r \approx \epsilon_f\frac{\mid{f(x)}\mid}{h}$ 

where $\epsilon_f$ is the fractional accuracy with which $f$ is computed.  This is generally machine accuracy

$\epsilon_f = \mbox{np.finfo(float).eps}$

The remainder that we showed earlier, $\displaystyle h^2 \frac{f'''(\epsilon_1)+ f'''(\epsilon_2)}{12}$ is referred to as **truncation** error.

Minimizing 

$\displaystyle \epsilon_r  + \epsilon_t \approx \epsilon_f\frac{\mid{f(x)}\mid}{h} + h^2 \frac{f'''(\epsilon_1)+ f'''(\epsilon_2)}{12}$

we obtain 

$\displaystyle h \sim \epsilon_f^{1/3} \left(\frac{f}{f'''}\right)^{1/3}$

where $\displaystyle \left(\frac{f}{f'''}\right)^{1/3}$ is shorthand for the ratio of $f(x)$ and the sum of $f'''(\epsilon_1)+ f'''(\epsilon_2)$.

We use shorthand here because because we are not going to approximate $f'''$ (we are already approximating $f'$), so there is no point in writing it out.  

Call the shorthand 

$\displaystyle \left(\frac{f}{f'''}\right)^{1/3}=x_c$

the curvature scale, or characteristic scale of the function $f$.

There are several algorithms for choosing an optimal scale.  

The better the scale chosen, the more accurate the approximation is.

A good rule of thumb, which is computationally quick, is to just use the absolute value of $x$.

$x_c = \mid{x}\mid$

Then we would use

$h = \epsilon_f^{1/3} \mid{x}\mid$

But what if $x$ is 0?

This is simple to handle, we just add $\epsilon_f^{1/3}$ to $x_c = \mid x \mid$

$h = \epsilon_f^{1/3} ( \mid{x}\mid + \epsilon_f^{1/3})$

Now, Press et al. also suggest performing a final sequence of assignment operations that ensures $x$ and $x+h$ differ by an exactly representable number.  You assign $x+h$ to a temporary variable $temp$.  Then $h$ is assigned the value of $temp-h$.

Using some dummy values for $x$ and $h$, the code would look like

In [1]:
x=32
h=.5

In [2]:
temp = x + h
h = temp - x

## Estimating Variance after Optimization

Now let's apply what we have learned to estimating $H(\widehat{\boldsymbol \theta})^{-1}$ and $\frac{\delta}{\delta \widehat{\boldsymbol{\theta}}} g(\widehat{\boldsymbol{\theta}})$.

Pystan will provide an estimate of the constrained posterior mode $g(\widehat{\boldsymbol \theta})$ by using the **optimizing()** function.  We'll refer to this value as **opt_parameters**. 

Now to get started on estimating $H(\widehat{\boldsymbol \theta})^{-1}$ we have to call PyStan's **sampling()** function.    

We use 1 iteration and specify **[opt_parameters]** in the **init** option and use "Fixed_param" in the **algorithm** option.  Let's call the object that **sampling()** returns **samp**.

This will allow us to compute the probability and gradient for the unconstrained posterior mode $\widehat{\boldsymbol \theta}$ .

We get the unconstrained posterior mode, $\widehat{\boldsymbol \theta}$ by using the **unconstrain_pars()** function with argument **opt_parameters** on **samp**.  We refer to this value as **unconstrained_pars**.   

The **grad_log_prob()** function can be applied to **samp** with argument **unconstrained_pars** to get the score function for the unconstrained parameters.

The **constrain_pars()** function can be used to compute the constrained parameters for any given input unconstrained parameters.

We loop over the unconstrained parameters $\theta_1,\ldots,\theta_K$.  

For iteration $i$, we calculate the bandwidth $h_i$ for parameter $\theta_i$, as discussed in the previous section with $x=\theta_i$.  We also create two copies of the **unconstrained_pars** vector, **parm_plus** and **parm_minus**.

**parm_plus** is updated by adding $h_i$ to $\theta_i$ and **parm_minus** is updated by subtracting $h_i$ from $\theta_i$.
Then we use the **grad_log_prob()** function to compute the score at **parm_plus** and the score at **parm_minus**.  
These are stored in **grad_plus** and **grad_minus**.  Column $i$ of the Hessian is computed as (**grad_plus**-**grad_minus**)/2$h_i$.  

We use the **constrain_pars** function to compute the constrained parameters at **parm_plus** and the constrained parameters at **par_minus**.  These are stored as **cons_plus** and **cons_minus**.  Column $i$ of $\frac{\delta}{\delta \widehat{\boldsymbol{\theta}}} g(\widehat{\boldsymbol{\theta}})$ is computed as (**cons_plus**-**cons_minus**)/2$h_i$.  

After looping over the parameters, we compute the matrix product $\left\{\frac{\delta}{\delta \widehat{\boldsymbol{\theta}}} g(\widehat{\boldsymbol{\theta}})\right\}^T \left\{-H(\widehat{\boldsymbol{\theta}})^{-1}\right\} \left\{\frac{\delta}{\delta \widehat{\boldsymbol{\theta}}} g(\widehat{\boldsymbol{\theta}})\right\}$, our estimate of the covariance of $\boldsymbol{\theta_{c}}= g(\boldsymbol \theta)$.


Let's define a Python function that will get this done.  This functionality may be broken into several functions later, and customization in the optimization will be provided.

In [3]:
import pystan
import numpy as np

In [4]:
def estimate_map(stan_model,stan_data):
    sm = stan_model
    optimizing_fit = sm.optimizing(stan_data,algorithm="Newton",init='0')
    opt_parameters = optimizing_fit
    parkeys = list(opt_parameters.keys())
    test_grad_fit = sm.sampling(stan_data,              \
        iter=1,chains=1,warmup=0,init=[opt_parameters],check_hmc_diagnostics=False, \
                                algorithm="Fixed_param")
    unconstrained_pars = test_grad_fit.unconstrain_pars(opt_parameters)
    nparm = unconstrained_pars.shape[0]
    # note that constrained and unconstrained might have different dimensions.
    nparmc = len(parkeys)
    pars = unconstrained_pars
    

    Hessian = 0*np.ones((nparm,nparm))
    Delta = 0*np.ones((nparmc,nparm))
    epsdouble = np.finfo(float).eps
    epsdouble = epsdouble**(1/3)
    for i in np.arange(0,nparm):
        scaleparm = abs(pars[i])
        scale = epsdouble*(scaleparm+epsdouble)
        scaleparmstable = scaleparm+scale
        scale = scaleparmstable-scaleparm
        parmplus = pars.copy()
        parmminus = pars.copy()
        parmplus[i]=parmplus[i]+scale
        parmminus[i]=parmminus[i]-scale
        gradplus = test_grad_fit.grad_log_prob(parmplus)
        gradminus = test_grad_fit.grad_log_prob(parmminus)
        Hessian[:,i] = (gradplus-gradminus)/(2*scale)
        consplus = test_grad_fit.constrain_pars(parmplus)
        consminus = test_grad_fit.constrain_pars(parmminus)
        Delta[:,i] = (consplus[0:nparmc]-consminus[0:nparmc])/(2*scale)

    iHessian = -np.linalg.inv(Hessian)
    Cov = iHessian
    Cov = np.matmul(np.matmul(Delta,Cov),Delta)
    return {'mode':opt_parameters,'Cov':Cov,"stan_model":sm}

We can provide users standard deviations based on this estimated covariance.  This will provide measures of variability for the parameters.

## Estimating Confidence Intervals after Optimization

With the posterior mode, variance, and normal approximation to the posterior.  It is simple to create confidence (credible) intervals for the parameters.

Let's talk a little bit about what these intervals are.

For the parameter $\gamma$ we want a $(1-\alpha)%$ interval $(u,l)$ (defined on the observed data generated by a realization of $\gamma$) to be defined such that

$\displaystyle \mbox{Pr}(\gamma \in (u,l)) = 1-\alpha$

The frequentist confidence interval does not meet this criteria.  $\gamma$ is just one fixed value, so it is either in the interval, or it isn't!

A credible interval (Bayesian confidence interval) can meet this criteria.

Suppose that we are able to use the normal approximation for $\gamma \vert \bf{x}$

$\displaystyle \gamma\vert{\bf{x}} \approx_D N(\hat\gamma,\hat\sigma_\gamma^2)$

$\displaystyle \begin{align}
1-\alpha &= \mbox{Pr}(l  \leq \gamma \leq u \vert {\bf x}) \\
&= \mbox{Pr}(l - \hat{\gamma} \leq \gamma - \hat{\gamma} \leq u - \hat{\gamma} \vert {\bf x}) \\
&= \mbox{Pr}\left(\frac{l - \hat{\gamma}}{\hat\sigma_\gamma} \leq \frac{\gamma - \hat{\gamma}}{\hat\sigma_\gamma} \leq \frac{u - \hat{\gamma}}{\hat\sigma_\gamma} \vert {\bf x}\right) \\
\end{align}$

Now, $\displaystyle \frac{\gamma - \hat{\gamma}}{\hat\sigma_\gamma^2}$  is $N(0,1)$, standard normal.  So we can use the standard normal quantiles in solving for $l$ and $u$.   

The upper $\alpha/ 2$ quantile of the standard normal distribution, $z_{\alpha/ 2}$ satisfies

$\displaystyle \mbox{Pr}(Z \geq z_{\alpha/ 2}) = \alpha / 2$ 

for standard normal $Z$.

Noting that the standard normal is symmetric, if we can find $l$ and $u$ to satisfy

$\displaystyle \frac{l - \hat{\gamma}}{\hat\sigma_\gamma} = - z_{\alpha/ 2}$

$\displaystyle \frac{u - \hat{\gamma}}{\hat\sigma_\gamma} = z_{\alpha/ 2}$

then we have a valid Bayesian confidence interval.

Simple calculation shows that the solutions are

$\displaystyle l = - z_{\alpha/ 2}\hat\sigma_\gamma + \hat{\gamma}$ 

$\displaystyle u = z_{\alpha/ 2}\hat\sigma_\gamma + \hat{\gamma}$ 

The $z_{\alpha/ 2}$ quantile can be easily generated using **scipy.stats**.

We can also adjust the intervals for inference on many parameters by using Bonferroni correction.

## Example

Now we know how to estimate the posterior variance after computing the posterior mode.  We also know how confidence intervals are made based on this posterior variance, mode, and the normal approximation to the posterior. 

Let's perform a simulation to corroborate our results and show how they can be used in practice.

We will see that we can properly estimate the posterior mode and variance, and use the normal approximation for the posterior distribution to generate confidence intervals.

First we import the necessary libraries.

In [5]:
import pystan
import numpy as np
import pandas as pd
import scipy.stats as spstats

Now let's define our Stan specification for the model.  This a linear regression with a $\chi^2$ prior for the intercept $\alpha$ and a normal prior for the slope $\beta$.

In [6]:
model = """
data {
    int<lower=0> N;
    vector[N] x;
    vector[N] y;
}

parameters {
    real<lower=0> alpha;
    real beta;
}

model {
    alpha ~ chi_square(4);
    beta ~ normal(1,1);
    y ~ normal(alpha + beta * x,1);
}
"""
stan_model = pystan.StanModel(model_code=model)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_a0d1a455dfb70045e204be5bae7f04b9 NOW.


We have a constrained parameter and different types of prior distributions here.  This should provide robust corroboration that our methods work.  Note again that the priors are proper.  This is a requirement for our asymptotic approximations.

Now let's define a simulation function that we can call to draw from the model.  It takes as argument the sample size **n** and a random state **rs**.

In [7]:
def draw(n,rs):
     
    # draw parameters from priors
    beta = rs.normal(1.0,1)
    alpha = rs.chisquare(4)
    # draw observed data based on drawn parameters
    x = rs.normal(size=n)
    y = alpha + beta * x
    y = rs.normal(y,1)
    data = {'N': n, 'x': x, 'y': y}
    return (data)

Once we obtain a simulated draw from the model, we want to check the distribution of the posterior.  

If we perform multiple draws from the original model and evaluate them marginally over the draws, we **are** evaluating the posterior.  

As we demonstrated in previous notebooks, this can help us determine misspecification of the posterior. 

It will not let us directly evalute the posterior distribution though, because we are averaging over multiple draws of the observed data.

So for each draw from the original model, we will use MCMC to perform posterior draws, which will then be used to evaluate
the posterior distribution.

The draws provide a posterior sample of $\alpha$ and $\beta$. 

As mentioned in previous notebooks, the posterior sample may need to be thinned to reduce autocorrelation and obtain an independent sample.

Once thinning is completed, we create another parameter, $s_{\alpha\beta} = \alpha + \beta$.  


For each posterior sample, we record the mean and standard deviation of $\alpha$, $\beta$, and $s_{\alpha\beta}$ as $\mu_\alpha$, $\mu_\beta$, and $\mu_{s_{\alpha\beta}}$ and $\sigma_\alpha$, $\sigma_\beta$, and $\sigma_{s_{\alpha\beta}}$.

The standard deviation of $s_{\alpha\beta}$, $\sigma_{s_{\alpha\beta}}$ is the square-root of  

$\mbox{Var}(\alpha+\beta) = \mbox{Var}(\alpha) + \mbox{Var}(\beta) + 2 \mbox{Cov}(\alpha,\beta)$

So evaluating the standard deviation of $s_{\alpha\beta}$ will help us evaluate the covariance of $\alpha$ and $\beta$.

We also record our estimates of the posterior mode and variance that we derived using numerical optimization and the approximation methods discussed above.  These are $\hat\mu_\alpha$, $\hat\mu_\beta$, and $\hat\mu_{s_{\alpha\beta}}$ and $\hat\sigma_\alpha$, $\hat\sigma_\beta$, and $\hat\sigma_{s_{\alpha\beta}}$. 

Note that the mean and mode of the posterior distribution should be equal if it is normal.  

So it is reasonable to use $\mu$ to denote both.

If our estimates are accurate, and the normal approximation holds, then we should have the following

$\displaystyle \frac{\alpha-\hat\mu_\alpha}{\hat\sigma_\alpha}\sim N(0,1)$

$\displaystyle \frac{\beta-\hat\mu_\beta}{\hat\sigma_\beta}\sim N(0,1)$

$\displaystyle \frac{s_{\alpha\beta}-\hat\mu_{s_{\alpha\beta}}}{\hat\sigma_{s_{\alpha\beta}}}\sim N(0,1)$

We can check this assumption using an Anderson-Darling test. We store a record of whether the test rejects normality at the .05 significance level.  These rejection indicators are $r_\alpha$, $r_\beta$, and $r_{s_{\alpha\beta}}$. 


We also generate 95% confidence intervals for each parameter.  We will check the coverage of the intervals by recording the fraction of the thinned MCMC samples that are cotained within.  This will also check the normal approximation.

Using the methods discussed in the last section, the interval endpoints are

$\displaystyle l_\alpha = - z_{.025}\hat\sigma_\alpha + \hat\mu_{\alpha}$ 

$\displaystyle u_\alpha = z_{.025}\hat\sigma_\alpha + \hat\mu_{\alpha}$ 

$\displaystyle l_\beta = - z_{.025}\hat\sigma_\beta + \hat\mu_{\beta}$ 

$\displaystyle u_\beta = z_{.025}\hat\sigma_\beta + \hat\mu_{\beta}$ 

$\displaystyle l_{s_{\alpha\beta}} = - z_{.025}\hat\sigma_{s_{\alpha\beta}} + \hat\mu_{{s_{\alpha\beta}}}$ 

$\displaystyle u_{s_{\alpha\beta}} = z_{.025}\hat\sigma_{s_{\alpha\beta}} + \hat\mu_{{s_{\alpha\beta}}}$ 

The following function performs the posterior sampling and returns the results.  **L** denotes the number of posterior samples to draw.

In [8]:
def posterior_results(mode,Cov,stan_model,stan_data,L,rs):
    samptest_grad_fit = stan_model.sampling(stan_data,              \
        iter=3000,chains=1,warmup=1000,init=[mode],seed=rs,check_hmc_diagnostics=False)        
    summary = pd.DataFrame(samptest_grad_fit.summary()['summary'], \
                           columns=samptest_grad_fit.summary()['summary_colnames'])
    n_eff = np.min(summary['n_eff'])
    N = samptest_grad_fit.sim["iter"]
    N_skip = np.array([np.ceil(N / n_eff)]).astype(int)[0]
    N_needed = N_skip * L + samptest_grad_fit.sim["warmup"]
    if N_needed > N:
            samptest_grad_fit = stan_model.sampling(stan_data,              \
                iter=N_needed,chains=1,warmup=1000,init=[mode],seed=rs,check_hmc_diagnostics=False)        

    df = samptest_grad_fit.extract()
    sampalpha  = np.array(df['alpha'][::(N_skip)])
    sampalpha = sampalpha[0:L]
    sampbeta  = np.array(df['beta'][::(N_skip)])
    sampbeta = sampbeta[0:L]
    sampsum = sampalpha + sampbeta
    meanalpha = np.mean(sampalpha)
    meanbeta = np.mean(sampbeta)
    meansum = np.mean(sampsum)
    sealpha = np.std(sampalpha)
    sebeta = np.std(sampbeta)
    sesum = np.std(sampsum)
    adalpha = spstats.anderson((sampalpha-mode['alpha'])/np.sqrt(Cov[0,0]),'norm')
    adalpha = adalpha[1][2] < adalpha[0]
    adbeta = spstats.anderson((sampbeta-mode['beta'])/np.sqrt(Cov[1,1]),'norm')
    adbeta = adbeta[1][2] < adbeta[0]
    
    sesumhat  = np.sqrt(np.matmul(np.ones((1,2)),np.matmul(Cov,np.ones((2,1)))))
    sumest = (sampsum-  (mode['alpha'] + mode['beta']))/sesumhat
    sumest = sumest.flatten()
    adsum = spstats.anderson(sumest,'norm')
    adsum = adsum[1][2] < adsum[0]
    
    mean = np.array([meanalpha,meanbeta,meansum])
    se = np.array([sealpha,sebeta,sesum])
    ad = np.array([adalpha,adbeta,adsum])
    
    z025 = spstats.norm.ppf(1-.025)
    
    lalpha = -z025*np.sqrt(Cov[0,0]) + mode['alpha']
    ualpha = z025*np.sqrt(Cov[0,0]) + mode['alpha']
    lbeta = -z025*np.sqrt(Cov[1,1]) + mode['beta']
    ubeta = z025*np.sqrt(Cov[1,1]) + mode['beta']
    lsum = -z025*sesumhat + mode['alpha'] + mode['beta']
    usum = z025*sesumhat + mode['alpha'] + mode['beta']
    propalpha = np.mean(np.less_equal(sampalpha,ualpha)*np.greater_equal(sampalpha,lalpha))
    propbeta = np.mean(np.less_equal(sampbeta,ubeta)*np.greater_equal(sampbeta,lbeta))
    propsum = np.mean(np.less_equal(sampsum,usum)*np.greater_equal(sampsum,lsum))
    prop = np.array([propalpha,propbeta,propsum])         
    return {'mu':mean, 'sigma': se, 'reject': ad, 'prop': prop}

Now let's perform the simulation.  We use an observed data sample size of **n=600** based on **draws=1000** prior draws.  For each observed data sample we will use **L=500** posterior draws to evaluate the confidence intervals and posterior normal approximation.  We store the results in the **results** matrix.

In [9]:
n = 600
draws = 1000
L=500
rs = np.random.RandomState(218409)
results = 0*np.ones((draws,18))

In [10]:
for i in np.arange(0, draws):
    print(i)
    data = draw(n,rs)
    est_fit = estimate_map(stan_model,data)
    post = posterior_results(est_fit['mode'],est_fit['Cov'],est_fit['stan_model'],data,L,rs)
    results[i,0]= est_fit['mode']['alpha']
    results[i,1] = post['mu'][0]
    results[i,2] = np.sqrt(est_fit['Cov'][0,0])
    results[i,3] = post['sigma'][0]

    results[i,4]= est_fit['mode']['beta']
    results[i,5] = post['mu'][1]
    results[i,6] = np.sqrt(est_fit['Cov'][1,1])
    results[i,7] = post['sigma'][1]
    
    results[i,8]= est_fit['mode']['alpha'] + est_fit['mode']['beta']
    results[i,9] = post['mu'][2]
    results[i,10] = np.sqrt(np.matmul(np.ones((1,2)),np.matmul(est_fit['Cov'],np.ones((2,1)))))
    results[i,11] = post['sigma'][2]
    
    results[i,12] = post['reject'][0]
    results[i,13] = post['reject'][1]
    results[i,14] = post['reject'][2]
    results[i,15] = post['prop'][0]
    results[i,16] = post['prop'][1]
    results[i,17] = post['prop'][2]
    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27


Now we summarize the results.

In [11]:
ResultSummary = pd.DataFrame((np.concatenate(( \
                                np.mean(results,axis=0)[:,np.newaxis], \
                                np.std(results,axis=0)[:,np.newaxis]), \
                                             axis=1)),index=[          \
    "Alpha Mode","Alpha Mean Posterior", "SE Alpha Estimate","Alpha SD Posterior", \
    "Beta Mode","Beta Mean Posterior", "SE Beta Estimate","Beta SD Posterior", \
    "Sum Mode","Sum Mean Posterior", "SE Sum Estimate","Sum SD Posterior", \
     "Alpha AD Reject Rate", "Beta AD Reject Rate","Sum AD Reject Rate", \
     "Alpha CI Proportion", "Beta CI Proportion", "Sum CI Proportion"], \
                                             columns=['Mean',"SD"])
ResultSummary

,Mean,SD
Alpha Mode,4.170080,3.073904
Alpha Mean Posterior,4.170141,3.073850
SE Alpha Estimate,0.040828,0.000207
Alpha SD Posterior,0.040723,0.001481
Beta Mode,0.996798,1.009497
Beta Mean Posterior,0.996799,1.009507
SE Beta Estimate,0.040869,0.001188
Beta SD Posterior,0.040815,0.001952
Sum Mode,5.166878,3.236201
Sum Mean Posterior,5.166940,3.236160


The modes are close the posterior means on average.  The same is true for the posterior standard deviation and estimated standard error.

The rejection rates are close to .05, and the proportions of the samples in the 95% confidence intervals are close to .95  We can perform a hypothesis test of whether the rejection rate is .05 by checking whether .05 is in the confidence interval for the proportion.  We can test whether the proportions are .95 in an analogous manner.

In [15]:
import statsmodels.stats.proportion as smp

lower, upper = smp.proportion_confint(np.sum(results[:,12:],axis=0), \
                                      draws, method='beta')
CiResults = pd.DataFrame(np.concatenate((lower[:,np.newaxis], \
                                         upper[:,np.newaxis]),axis=1), \
                                         index=["Alpha AD Reject Rate", "Beta AD Reject Rate","Sum AD Reject Rate", \
     "Alpha CI Proportion", "Beta CI Proportion", "Sum CI Proportion"], \
                                         columns = ('Lower', 'Upper'))
CiResults

,Lower,Upper
Alpha AD Reject Rate,0.044333,0.074336
Beta AD Reject Rate,0.038205,0.066513
Sum AD Reject Rate,0.039077,0.067635
Alpha CI Proportion,0.935145,0.963078
Beta CI Proportion,0.934250,0.962386
Sum CI Proportion,0.934738,0.962764


.05 is in all the rejection rate intervals.  .95 is in the all of the confidence interval proportion intervals.

Our simulation was successful.